In [ ]:
import numpy as np
import pandas as pd
import pymc as pm
import matplotlib.pyplot as plt
from io import StringIO

import os
os.environ["PYTENSOR_FLAGS"] = "cxx=;linker=py"


# ============================================================
# 1) データ（架空の株価 60本）
# ============================================================
csv_data = """date,close
2025-11-03,100.63
2025-11-04,100.49
2025-11-05,101.31
2025-11-06,103.21
2025-11-07,102.95
2025-11-10,102.72
2025-11-11,104.73
2025-11-12,105.73
2025-11-13,105.18
2025-11-14,105.90
2025-11-17,105.37
2025-11-18,104.85
2025-11-19,105.19
2025-11-20,102.91
2025-11-21,100.82
2025-11-24,100.17
2025-11-25,99.00
2025-11-26,99.41
2025-11-27,98.36
2025-11-28,96.72
2025-12-01,98.17
2025-12-02,97.93
2025-12-03,98.04
2025-12-04,96.39
2025-12-05,95.79
2025-12-08,95.93
2025-12-09,94.68
2025-12-10,95.15
2025-12-11,94.49
2025-12-12,94.19
2025-12-15,93.49
2025-12-16,95.12
2025-12-17,95.12
2025-12-18,93.86
2025-12-19,94.31
2025-12-22,92.95
2025-12-23,91.61
2025-12-24,91.93
2025-12-25,90.88
2025-12-26,91.18
2025-12-29,92.17
2025-12-30,92.42
2025-12-31,92.30
2026-01-01,91.28
2026-01-02,90.17
2026-01-05,89.61
2026-01-06,89.99
2026-01-07,90.40
2026-01-08,89.32
2026-01-09,89.73
2026-01-12,89.45
2026-01-13,89.65
2026-01-14,90.13
2026-01-15,90.17
2026-01-16,91.43
2026-01-19,90.81
2026-01-20,89.92
2026-01-21,89.62
2026-01-22,90.00
2026-01-23,91.09
"""
df = pd.read_csv(StringIO(csv_data))
y = df["close"].to_numpy().astype(float)

y_prev = y[:-1]
y_next = y[1:]
y0 = y[0]

# ============================================================
# 2) PyMC: AR(1) 尤度を手書き（pm.ARを使わない）
#    y_t ~ Normal( mu + phi*(y_{t-1}-mu), sigma )
# ============================================================
with pm.Model() as model:
    mu = pm.Normal("mu", mu=np.mean(y), sigma=50)
    phi = pm.Uniform("phi", lower=-0.99, upper=0.99)   # 安定のため範囲を制限
    sigma = pm.Exponential("sigma", 1.0)

    # 初期値のゆるいモデル（ここは簡易）
    pm.Normal("y0_like", mu=mu, sigma=10*sigma, observed=y0)

    # AR(1) 本体
    pm.Normal(
        "y_like",
        mu=mu + phi * (y_prev - mu),
        sigma=sigma,
        observed=y_next
    )

    trace = pm.sample(
        draws=1000,
        tune=1000,
        target_accept=0.9,
        chains=2,
        cores=1
    )

# ============================================================
# 3) 未来10日の予測（事後サンプルからシミュレーション）
# ============================================================
posterior = trace.posterior

mu_samps = posterior["mu"].stack(s=("chain", "draw")).values
phi_samps = posterior["phi"].stack(s=("chain", "draw")).values
sigma_samps = posterior["sigma"].stack(s=("chain", "draw")).values

def forecast_ar1(y_last, mu, phi, sigma, steps=10):
    x = y_last
    out = []
    for _ in range(steps):
        x = np.random.normal(mu + phi*(x - mu), sigma)
        out.append(x)
    return out

steps = 10
n = len(mu_samps)
future = np.empty((n, steps), dtype=float)

for i in range(n):
    future[i] = forecast_ar1(y_last=y[-1], mu=mu_samps[i], phi=phi_samps[i], sigma=sigma_samps[i], steps=steps)

mean_pred = future.mean(axis=0)
p5 = np.percentile(future, 5, axis=0)
p95 = np.percentile(future, 95, axis=0)

# ============================================================
# 4) プロット（平均 + 5-95%）
# ============================================================
days = np.arange(1, steps+1)

plt.figure(figsize=(10, 5))
plt.plot(days, mean_pred, label="mean forecast")
plt.fill_between(days, p5, p95, alpha=0.3, label="5–95% interval")
plt.title("Bayesian AR(1) via MCMC (next 10 days)")
plt.xlabel("future day")
plt.ylabel("price")
plt.legend()
plt.grid(True)
plt.show()


: 